# Анализ и обработка пропусков в данных

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np
from pprint import pprint, pformat
import copy
import zipfile
from pathlib import Path


import missingno as msno
import joblib

from my_lib import *
from my_config import *

In [2]:
dataset_df = joblib.load(Path(result_foler, dataset_filename_01_after_EDA))
params = joblib.load(Path(result_foler, params_filename_01_after_EDA))
display(dataset_df)
display(params)

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
id,,,,,,,,,,,,,,,,,,,,
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,Premium,NaN,5.0,372.0,3.0,2023-05-03 15:21:39.257696,Poor,No,Daily,Apartment,1303.0
1199996,54.0,Male,35786.0,Divorced,NaN,Master's,Self-Employed,11.483482,Rural,Comprehensive,NaN,10.0,597.0,4.0,2022-09-10 15:21:39.134960,Poor,No,Weekly,Apartment,821.0
1199997,19.0,Male,51884.0,Divorced,0.0,Master's,NaN,14.724469,Suburban,Basic,0.0,19.0,NaN,6.0,2021-05-25 15:21:39.106582,Good,No,Monthly,Condo,371.0


{'target_columns': 'Premium Amount',
 'columns_X': ['Age',
  'Gender',
  'Annual Income',
  'Marital Status',
  'Number of Dependents',
  'Education Level',
  'Occupation',
  'Health Score',
  'Location',
  'Policy Type',
  'Previous Claims',
  'Vehicle Age',
  'Credit Score',
  'Insurance Duration',
  'Policy Start Date',
  'Customer Feedback',
  'Smoking Status',
  'Exercise Frequency',
  'Property Type'],
 'cat_columns': ['Gender',
  'Marital Status',
  'Education Level',
  'Occupation',
  'Location',
  'Policy Type',
  'Policy Start Date',
  'Customer Feedback',
  'Smoking Status',
  'Exercise Frequency',
  'Property Type'],
 'num_columns': ['Age',
  'Annual Income',
  'Number of Dependents',
  'Health Score',
  'Previous Claims',
  'Vehicle Age',
  'Credit Score',
  'Insurance Duration']}

In [3]:
print("Доля пропусков в столбцах в %:")
nan_in_columns = dataset_df.isna().sum()/len(dataset_df)*100
# список столбцов с пропусками
params["columns_with_nan"] = nan_in_columns[nan_in_columns > 0].index.to_list()
nan_in_columns[nan_in_columns > 0]

Доля пропусков в столбцах в %:


Age                      1.558750
Annual Income            3.745750
Marital Status           1.544083
Number of Dependents     9.139333
Occupation              29.839583
Health Score             6.173000
Previous Claims         30.335750
Vehicle Age              0.000500
Credit Score            11.490167
Insurance Duration       0.000083
Customer Feedback        6.485333
dtype: float64

In [4]:
# исключим из рассмотрения столбцы, в которых более 10% пропусков
for col in nan_in_columns[nan_in_columns > 10].index:
    print(f'{col} исключен из рассмотрения')
    remove_columns(col, params)
pprint(params)    

Occupation исключен из рассмотрения
Previous Claims исключен из рассмотрения
Credit Score исключен из рассмотрения
{'cat_columns': ['Gender',
                 'Marital Status',
                 'Education Level',
                 'Location',
                 'Policy Type',
                 'Policy Start Date',
                 'Customer Feedback',
                 'Smoking Status',
                 'Exercise Frequency',
                 'Property Type'],
 'columns_X': ['Age',
               'Gender',
               'Annual Income',
               'Marital Status',
               'Number of Dependents',
               'Education Level',
               'Health Score',
               'Location',
               'Policy Type',
               'Vehicle Age',
               'Insurance Duration',
               'Policy Start Date',
               'Customer Feedback',
               'Smoking Status',
               'Exercise Frequency',
               'Property Type'],
 'columns_with_nan': ['Age

In [5]:
# посчитать сколько столбцов с пропусками в строках
dataset_df['nan_count'] = dataset_df[params["columns_with_nan"]].apply(lambda x: x.isna().sum(), axis=1)  

In [6]:
#display(dataset_df.sample(7))
limit_nan_count = 1
display(dataset_df[dataset_df['nan_count'] == limit_nan_count][params["columns_with_nan"]])
rows_with_nan_count = len(dataset_df[dataset_df['nan_count'] >= limit_nan_count])
all_rows_count = len(dataset_df)
print(f"Доля строк с {limit_nan_count} пропусками: {rows_with_nan_count} из {all_rows_count} ({round(rows_with_nan_count/all_rows_count*100,2)}%)")

,Age,Annual Income,Marital Status,Number of Dependents,Health Score,Vehicle Age,Insurance Duration,Customer Feedback
id,,,,,,,,
6,41.0,40336.0,Married,0.0,NaN,8.0,6.0,Poor
10,56.0,8054.0,Married,1.0,NaN,8.0,8.0,Average
14,40.0,23897.0,Divorced,NaN,29.082036,15.0,1.0,Good
18,18.0,29071.0,Married,NaN,20.838977,5.0,3.0,Poor
22,22.0,NaN,Divorced,4.0,25.583790,5.0,5.0,Good
...,...,...,...,...,...,...,...,...
1199987,25.0,4089.0,Single,1.0,12.335586,17.0,1.0,NaN
1199989,18.0,29092.0,Single,2.0,NaN,14.0,1.0,Poor
1199996,54.0,35786.0,Divorced,NaN,11.483482,10.0,4.0,Poor


Доля строк с 1 пропусками: 306817 из 1200000 (25.57%)


In [7]:
# сформируем датасет без строк с пропусками в столбцах
dataset_df_wo_nan = dataset_df[dataset_df["nan_count"]==0].copy(deep=True)
# сбросим индекс
dataset_df_wo_nan.reset_index(drop=True, inplace=True)
display(dataset_df_wo_nan.sample(7))

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,...,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount,nan_count
757250,32.0,Male,66780.0,Married,3.0,Bachelor's,Self-Employed,43.903132,Urban,Basic,...,18.0,449.0,5.0,2022-01-08 15:21:39.192212,Average,No,Daily,Condo,749.0,0
650227,20.0,Male,32545.0,Married,3.0,Bachelor's,Self-Employed,38.533924,Rural,Comprehensive,...,19.0,315.0,3.0,2022-08-07 15:21:39.155231,Average,No,Weekly,House,1896.0,0
395928,52.0,Female,10360.0,Single,0.0,PhD,Unemployed,22.014487,Urban,Comprehensive,...,9.0,491.0,8.0,2020-09-11 15:21:39.271453,Good,Yes,Monthly,Condo,1444.0,0
332139,37.0,Male,18146.0,Married,3.0,Master's,NaN,26.846271,Urban,Basic,...,18.0,487.0,1.0,2022-11-24 15:21:39.092127,Average,Yes,Daily,Condo,724.0,0
1531,36.0,Female,16087.0,Divorced,1.0,High School,NaN,13.543264,Suburban,Basic,...,13.0,451.0,7.0,2019-10-04 15:21:39.278180,Poor,Yes,Rarely,Apartment,963.0,0
414027,22.0,Female,2907.0,Single,1.0,High School,Unemployed,30.387967,Rural,Basic,...,10.0,602.0,3.0,2023-02-01 15:21:39.236064,Average,No,Monthly,Condo,2589.0,0
389159,61.0,Male,1447.0,Single,2.0,Master's,Unemployed,28.143569,Urban,Basic,...,16.0,NaN,2.0,2022-05-08 15:21:39.281787,Good,No,Rarely,House,1290.0,0


In [8]:
print("Доля пропусков в столбцах в %:")
nan_in_columns = dataset_df_wo_nan[params["columns_X"]].isna().sum()/len(dataset_df_wo_nan)*100
# список столбцов с пропусками
columns_with_nan = nan_in_columns[nan_in_columns > 0].index.to_list()
nan_in_columns

Доля пропусков в столбцах в %:


Age                     0.0
Gender                  0.0
Annual Income           0.0
Marital Status          0.0
Number of Dependents    0.0
Education Level         0.0
Health Score            0.0
Location                0.0
Policy Type             0.0
Vehicle Age             0.0
Insurance Duration      0.0
Policy Start Date       0.0
Customer Feedback       0.0
Smoking Status          0.0
Exercise Frequency      0.0
Property Type           0.0
dtype: float64

In [9]:
# сохранить промежуточные результаты
joblib.dump(dataset_df, Path(result_foler, dataset_filename_02_after_PrepareGaps), compress=3)
joblib.dump(dataset_df_wo_nan, Path(result_foler, dataset_wo_nan_filename_02_after_PrepareGaps), compress=3)
joblib.dump(params, Path(result_foler, params_filename_02_after_PrepareGaps), compress=3)

['/home/igel/Projects/ml/ml-inno-hw/3. Machine Learning/3.03. Полиноминальная регрессия/result/03_params_after_PrepareGaps.joblib']